In [1]:
import os
import json
import wget 
import shutil
import numpy as np
import pandas as pd 

In [2]:
Home = os.getcwd()
DataFolder = os.path.join(Home,'Data')
df = pd.read_csv([os.path.join(DataFolder,i) for i in os.listdir(DataFolder) if i.endswith('.csv')][0])
df_cols = list(df.columns)
df = df.drop(columns=[i for i in df_cols if i.startswith('Unnamed:')])
#df.to_csv(os.path.join(DataFolder,'MyDataSet.csv'))

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

In [4]:
#df = pd.read_csv(DataFileName)
df_cols = list(df.columns)
y = df[df_cols[-1]]
X = df[df_cols[:-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = RandomForestClassifier()
model.fit(X_train,y_train)
val = model.score(X_test,y_test)
print(val)

0.909144305675776


In [11]:
from datetime import datetime as dt 

In [12]:
def Get_hms(val):
 
    sec = val/1000
    mini = sec/60 
    hr = mini/60
    ms = sec%1000
    hr %= 24
    mini %= 60
    sec %= 60 

    return int(hr), int(mini), int(sec) 
    

In [13]:
def Check_Time(df_timestamp,h,m,s):
    df_h,df_m,df_s = Get_hms(df_timestamp)
    #print(f'checking {df_h}:{df_m}:{df_s}')
    if df_h == h and df_m == m and df_s == s:
        return True
    return False


In [48]:
def Get_req_start_Idx(s_h,s_m,s_s,prev_idx,flag=True):
    df = pd.read_csv(energy_file)
    for i in range(prev_idx,len(df)):
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,s_h,s_m,s_s):
            return i
    if flag:
        print('re try')
        print(prev_idx)
        print(Get_hms(df['TimeStamp (ms)'][len(df)]))
        return Get_req_start_Idx(s_h,s_m,s_s,prev_idx=0,flag=False)
    else:
        return -1


In [49]:
def Cal_Energy(start_idx,e_h,e_m,e_s):
    #total_energy_lst = []
    total_energy = 0
    cpu_e = 0
    monitor_e = 0
    disk_e = 0
    base_e = 0
    df = pd.read_csv(energy_file)
#['TimeStamp (ms)', ' Total Power (W)', ' CPU (W)', ' Monitor (W)',' Disk (W)', ' Base (W)', ' Application (W)']
    for i in range(start_idx,len(df)):
        #total_energy_lst.append(df[' Total Power (W)'][i])
        total_energy += df[' Total Power (W)'][i]
        cpu_e += df[' CPU (W)'][i]
        monitor_e += df[' Monitor (W)'][i]
        disk_e += df[' Disk (W)'][i]
        base_e += df[' Base (W)'][i]
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,e_h,e_m,e_s):
            return total_energy,cpu_e,monitor_e,disk_e,base_e, i
    return total_energy,cpu_e,monitor_e,disk_e,base_e, len(df)

In [50]:
def Get_Time_Energy(start_time,end_time,prev_idx=0):

    s_d = start_time.day
    s_h = start_time.hour
    s_m = start_time.minute
    s_s = start_time.second
    #print('Start',s_d,s_h,s_m,s_s)
    e_d = end_time.day 
    e_h = end_time.hour
    e_m = end_time.minute
    e_s = end_time.second
    #print('End',e_d,e_h,e_m,e_s)
    time_taken = 0
    time_taken = 24*(time_taken + e_d-s_d)
    time_taken = 60*(time_taken + e_h-s_h)
    time_taken = 60*(time_taken + e_m - s_m)
    time_taken = (time_taken + e_s - s_s)

    start_idx = Get_req_start_Idx(s_h=s_h,s_m=s_m,s_s=s_s,prev_idx=prev_idx,flag=True)
    if start_idx == -1:
        print("FAILED TO FIND START IDX")
        return
    # total_energy,cpu_e,monitor_e,disk_e, i, df[' Base (W)'][0]
    total,cpu,monitor,disk,base,end_idx = Cal_Energy(start_idx,e_h,e_m,e_s)

    return total,cpu,monitor,disk,base, time_taken, end_idx


In [51]:
energy_file = [ os.path.join(os.getcwd(),x) for x in os.listdir() if x.endswith('.csv')][0]
energy_file


'c:\\Users\\chand\\Desktop\\work_simple\\Green-Computing\\temp\\energy_clerk.csv'

In [52]:
MyDict = dict()
My_prev_idx = 0
for M_type in range(5):
    print(M_type)
    for Para in range(50,501,50):
        print(Para)
        start_time = dt.now()
        Name, Score = Model_try(DataFile,M_type,Para)
        end_time = dt.now()
        #total,cpu,monitor,disk,base, time_taken, end_idx
        total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)
        TempDict = {'Total Energy (J)' : total, 'Total CPU  Energy (J)' : cpu, 'Total Monitor Energy (J)' : monitor, 'Total Disk Energy (J)' : disk, 'Base Energy (J)' : base, 'Time (Sec)' : time_taken, 'Score' : Score*100, 'n_estimators' : Para}
        if Name in MyDict:
            MyDict[Name].append(TempDict)
        else:
            MyDict[Name] = [TempDict]
        with open('Kaito Kids Dairy.json','w') as kid:
            json.dump(MyDict,kid)


0
50
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.702229480388142.
100
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7206528776141269.
150
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7172586324192401.
200
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7269577183886997.
250
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7243385725965737.
300
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7223028905669284.
350
Fitting at most 1 candidates
Optimization finished, 1 candidates were fitted
For Grey Wolf Optimizer we get a score of 0.7185203904364601.

In [17]:
import json 
import numpy as np 
import pandas as pd

In [18]:
MyDict = dict()
with open('Kaito Kids Dairy.json','r') as kid:
    MyDict = json.load(kid)

In [19]:
name_lst = []
TE_lst = []
CPU_lst = []
Mon_lst = []
Disk_lst = []
B_lst = []
Time_lst = []
Acc_lst = []
n_estimator_lst = []
for i in MyDict.keys():
    curr_lst = MyDict[i]
    for j in range(len(curr_lst)):
        inner_dict = curr_lst[j]
        
        name_lst.append(i)
        TE_lst.append(inner_dict['Total Energy (J)'])
        CPU_lst.append(inner_dict['Total CPU  Energy (J)'])
        Mon_lst.append(inner_dict['Total Monitor Energy (J)'])
        Disk_lst.append(inner_dict['Total Disk Energy (J)'])
        Time_lst.append(inner_dict['Time (Sec)'])
        B_lst.append(inner_dict['Base Energy (J)'])
        Acc_lst.append(inner_dict['Score'])
        n_estimator_lst.append(inner_dict['n_estimators'])


Final_dict = {'Algo Name':name_lst, 'n_estimators' : n_estimator_lst, 'Time Taken' : Time_lst, 'Accuracy' : Acc_lst,'Total Energy (J)':TE_lst,'Base Energy (J)' : B_lst,'CPU Energy (J)' : CPU_lst, 'Monitor Energy (J)' : Mon_lst, 'Disk Energy (J)' : Disk_lst }


df = pd.DataFrame(data=Final_dict)


In [20]:
try:
    os.mkdir('Result')
except:
    pass
df.to_csv('Result\\Full_Result.csv')

In [21]:
algo_gap = len(MyDict.keys())
iteration_gap = len(MyDict[list(MyDict.keys())[1]])
algo_gap,iteration_gap

(5, 10)

In [22]:
try:
    os.mkdir('Result\\Algo')
except:
    pass
try:
    os.mkdir('Result\\Estimator')
except:
    pass


In [23]:
for i in range(algo_gap):
    Name = Final_dict['Algo Name'][i*iteration_gap]
    TempDict = {'n_estimators' : n_estimator_lst[i*10:(i+1)*10], 'Time Taken' : Time_lst[i*10:(i+1)*10], 'Accuracy' : Acc_lst[i*10:(i+1)*10],'Total Energy (J)':TE_lst[i*10:(i+1)*10],'Base Energy (J)' : B_lst[i*10:(i+1)*10],'CPU Energy (J)' : CPU_lst[i*10:(i+1)*10], 'Monitor Energy (J)' : Mon_lst[i*10:(i+1)*10], 'Disk Energy (J)' : Disk_lst[i*10:(i+1)*10] }
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'Result\\Algo\\{Name}.csv')


In [24]:
for i in range(iteration_gap):
    estimators = Final_dict['n_estimators'][i]
    name_lst_temp = []
    TE_lst_temp = []
    CPU_lst_temp = []
    Mon_lst_temp = []
    Disk_lst_temp = []
    B_lst_temp = []
    Time_lst_temp = []
    Acc_lst_temp = []
    for j in range(algo_gap):
        name_lst_temp.append(name_lst[(j*iteration_gap)+i])
        TE_lst_temp.append(TE_lst[(j*iteration_gap)+i])
        CPU_lst_temp.append(CPU_lst[(j*iteration_gap)+i])
        Mon_lst_temp.append(Mon_lst[(j*iteration_gap)+i])
        Disk_lst_temp.append(Disk_lst[(j*iteration_gap)+i])
        B_lst_temp.append(B_lst[(j*iteration_gap)+i])
        Time_lst_temp.append(Time_lst[(j*iteration_gap)+i])
        Acc_lst_temp.append(Acc_lst[(j*iteration_gap)+i])

    TempDict = {'Algo Name' : name_lst_temp, 'Time Taken' : Time_lst_temp, 'Accuracy' : Acc_lst_temp,'Total Energy (J)':TE_lst_temp,'Base Energy (J)' : B_lst_temp,'CPU Energy (J)' : CPU_lst_temp, 'Monitor Energy (J)' : Mon_lst_temp, 'Disk Energy (J)' : Disk_lst_temp }
    
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'Result\\Estimator\\{estimators}.csv')


co2 switch

In [25]:
name_lst = []
TE_lst = []
CPU_lst = []
Mon_lst = []
Disk_lst = []
B_lst = []
Time_lst = []
Acc_lst = []
n_estimator_lst = []
for i in MyDict.keys():
    curr_lst = MyDict[i]
    for j in range(len(curr_lst)):
        inner_dict = curr_lst[j]
        
        name_lst.append(i)
        TE_lst.append((17.0/72.0)*inner_dict['Total Energy (J)'])
        CPU_lst.append((17.0/72.0)*inner_dict['Total CPU  Energy (J)'])
        Mon_lst.append((17.0/72.0)*inner_dict['Total Monitor Energy (J)'])
        Disk_lst.append((17.0/72.0)*inner_dict['Total Disk Energy (J)'])
        Time_lst.append(inner_dict['Time (Sec)'])
        B_lst.append((17.0/72.0)*inner_dict['Base Energy (J)'])
        Acc_lst.append(inner_dict['Score'])
        n_estimator_lst.append(inner_dict['n_estimators'])


Final_dict = {'Algo Name':name_lst, 'n_estimators' : n_estimator_lst, 'Time Taken' : Time_lst, 'Accuracy' : Acc_lst,'Total CO2 Emission (mg)':TE_lst,'Base CO2 Emission (mg)' : B_lst,'CPU CO2 Emission (mg)' : CPU_lst, 'Monitor CO2 Emission (mg)' : Mon_lst, 'Disk CO2 Emission (mg)' : Disk_lst }


df = pd.DataFrame(data=Final_dict)


In [26]:
try:
    os.mkdir('ResultCO2')
except:
    pass
df.to_csv('ResultCO2\\Full_Result.csv')

In [27]:
algo_gap = len(MyDict.keys())
iteration_gap = len(MyDict[list(MyDict.keys())[1]])
algo_gap,iteration_gap

(5, 10)

In [28]:
try:
    os.mkdir('ResultCO2\\Algo')
except:
    pass
try:
    os.mkdir('ResultCO2\\Estimator')
except:
    pass


In [29]:
for i in range(algo_gap):
    Name = Final_dict['Algo Name'][i*iteration_gap]
    TempDict = {'n_estimators' : n_estimator_lst[i*10:(i+1)*10], 'Time Taken' : Time_lst[i*10:(i+1)*10], 'Accuracy' : Acc_lst[i*10:(i+1)*10],'Total CO2 Emission (mg)':TE_lst[i*10:(i+1)*10],'Base CO2 Emission (mg)' : B_lst[i*10:(i+1)*10],'CPU CO2 Emission (mg)' : CPU_lst[i*10:(i+1)*10], 'Monitor CO2 Emission (mg)' : Mon_lst[i*10:(i+1)*10], 'Disk CO2 Emission (mg)' : Disk_lst[i*10:(i+1)*10] }
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'ResultCO2\\Algo\\{Name}.csv')


In [30]:
for i in range(iteration_gap):
    estimators = Final_dict['n_estimators'][i]
    name_lst_temp = []
    TE_lst_temp = []
    CPU_lst_temp = []
    Mon_lst_temp = []
    Disk_lst_temp = []
    B_lst_temp = []
    Time_lst_temp = []
    Acc_lst_temp = []
    for j in range(algo_gap):
        name_lst_temp.append(name_lst[(j*iteration_gap)+i])
        TE_lst_temp.append(TE_lst[(j*iteration_gap)+i])
        CPU_lst_temp.append(CPU_lst[(j*iteration_gap)+i])
        Mon_lst_temp.append(Mon_lst[(j*iteration_gap)+i])
        Disk_lst_temp.append(Disk_lst[(j*iteration_gap)+i])
        B_lst_temp.append(B_lst[(j*iteration_gap)+i])
        Time_lst_temp.append(Time_lst[(j*iteration_gap)+i])
        Acc_lst_temp.append(Acc_lst[(j*iteration_gap)+i])

    TempDict = {'Algo Name' : name_lst_temp, 'Time Taken' : Time_lst_temp, 'Accuracy' : Acc_lst_temp,'Total CO2 Emission (mg)':TE_lst_temp,'Base CO2 Emission (mg)' : B_lst_temp,'CPU CO2 Emission (mg)' : CPU_lst_temp, 'Monitor CO2 Emission (mg)' : Mon_lst_temp, 'Disk CO2 Emission (mg)' : Disk_lst_temp }
    
    temp_df = pd.DataFrame(data=TempDict)
    temp_df.to_csv(f'ResultCO2\\Estimator\\{estimators}.csv')


In [31]:
return

SyntaxError: 'return' outside function (3438313781.py, line 1)

Basement!

Don't go down there!!

There are only faulty or once used code chunks!!!

In [ ]:
score_lst = []
estimator_lst = []
para_lst = []
name_lst = []

print(score_lst[-1],estimator_lst[-1],para_lst[-1],name_lst[-1])

In [ ]:
score_lst = []
estimator_lst = []
para_lst = []
name_lst = []
MyDict = dict()
for i in range(5):
    ADict, NIA_Name = Model_try(DataFile,i)
    MyDict[NIA_Name] = ADict
    para_lst.append(ADict['Best Para'])
    score_lst.append(ADict['Best Score'])
    estimator_lst.append(ADict['Best Estimator'])
    name_lst.append(NIA_Name)


In [ ]:
k_lst = list(MyDict.keys())
k2_lst = list(MyDict[k_lst[0]].keys())
for i in k_lst:
  MyDict[i]['Best Estimator'] = None 

with open("NIA_Dicts.json", "w") as outfile:
    json.dump(MyDict,outfile) 

In [ ]:
df = pd.DataFrame(data= { 'NIA' : name_lst, 'Accuracy' : score_lst, 'Estimator' : estimator_lst, 'Para' : para_lst } )

In [ ]:
df.to_csv('Final Result.csv')

DON'T BE UP RUNNING IT PAST 00:00!!
YOU SHOULD BE IN BED BY THEN!!!!!!!

Here is a bed time story for you 


She carefully assured her child that there wasn't a monster under the bed, turned off the light and left.










Thanks to her, I now get to enjoy the feast I'd been craving for so long.
